In [ ]:
import pandas as pd
import numpy as np
import pickle
import time

In [ ]:
subject = "m292"

In [ ]:
label_list = [0, 1, 2, 3, 4]

In [ ]:
# Load persistence diagrams

persistence_diagrams = np.load("Embeddings_and_Persistence_Diagrams/"+str(subject)+"/Persistence_Diagrams.npy", allow_pickle=True).item()

In [ ]:
# Adcock-Carlson coordinates

In [ ]:
def carlsson_coordinates(persistence_diagrams):
    n = len(persistence_diagrams)
    persistence_diagrams_features_cc1 = np.zeros(n)
    persistence_diagrams_features_cc2 = np.zeros(n)
    persistence_diagrams_features_cc3 = np.zeros(n)
    persistence_diagrams_features_cc4 = np.zeros(n)
    start = time.time()
    ymax = 0
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            b = np.max(persistence_diagrams[i][:,1])
        else:
            b = ymax
        if ymax < b:
            ymax = b
        else:
            ymax = ymax
    print(ymax)
    for i in range(0,n):
        if len(persistence_diagrams[i])>0:
            x = persistence_diagrams[i][:,0]
            y = persistence_diagrams[i][:,1]
            persistence_diagrams_features_cc1[i] = sum(x*(y-x))
            persistence_diagrams_features_cc2[i] = sum((ymax - y)*(y-x))
            persistence_diagrams_features_cc3[i] = sum(x**2*(y-x)**4)
            persistence_diagrams_features_cc4[i] = sum((ymax-y)**2*(y-x)**4)
        else:
            persistence_diagrams_features_cc1[i] = 0
            persistence_diagrams_features_cc2[i] = 0
            persistence_diagrams_features_cc3[i] = 0
            persistence_diagrams_features_cc4[i] = 0
            
    return persistence_diagrams_features_cc1, persistence_diagrams_features_cc2, persistence_diagrams_features_cc3, persistence_diagrams_features_cc4

In [ ]:
def take_mean_coordinates(coord):
    
    amount_to_average = 80 # The imaging data has 20Hz, therefore we have to average 80 images to get 4 seconds

    averaged_coord = []
    # Take average of the statistics of 80 persistence diagrams 
    for idx in range(int(len(coord)/amount_to_average)):
        coord_sections = []
        
        for counter in range(amount_to_average):
            coord_sections.append(coord[amount_to_average*idx+counter].astype("float"))
        averaged_coord.append(np.mean(coord_sections, axis=0))

    return averaged_coord
    

In [ ]:
def create_dataframe(X_features_cc1, X_features_cc2, X_features_cc3, X_features_cc4, label):
    features_df = pd.DataFrame()

    features_df["AC_coordinate_cc1"] = X_features_cc1
    features_df["AC_coordinate_cc2"] = X_features_cc2
    features_df["AC_coordinate_cc3"] = X_features_cc3
    features_df["AC_coordinate_cc4"] = X_features_cc4
    features_df["Label"] = label

    return features_df

In [ ]:
features_dataframes = {}

for label in label_list:
    # Train data Adcock Carlson coordinates 
    coord_1, coord_2, coord_3, coord_4 = carlsson_coordinates(persistence_diagrams["Label_"+str(label)])
    coord_1 = take_mean_coordinates(coord_1)
    coord_2 = take_mean_coordinates(coord_2)
    coord_3 = take_mean_coordinates(coord_3)
    coord_4 = take_mean_coordinates(coord_4)
    features_dataframes[label] = create_dataframe(coord_1, coord_2, coord_3, coord_4, \
                                                        label)

In [ ]:
# Concatenate and save dataframes
feature_df = pd.concat([features_dataframes[0], features_dataframes[1], features_dataframes[2], \
                              features_dataframes[3], features_dataframes[4]], ignore_index=True)

feature_df.to_csv("Features/"+str(subject)+"/Advanced_Features.csv")